# crawling_irss

### 수집: 제목, 저자, 년도

### step 0. 준비

In [148]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

### step 1. 제목, 저자 크롤링

In [223]:
# 크롤링 할 검색어
keyword = input("크롤링할 검색어 입력: ")

크롤링할 검색어 입력: 통합 and 교수효능감


In [150]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"./chromedriver/chromedriver.exe")

In [13]:
# 사이트 주소
driver.get("http://www.riss.kr.libproxy.kongju.ac.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query={0}&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery={0}&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword={0}&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn=".format(keyword))
time.sleep(5)

In [88]:
# 년도 수동 클릭

In [224]:
title_list = []
name_list = []
year_list = []
kci_list = []

for i in range(3,100):
    # 제목 크롤링
    titles = driver.find_elements_by_css_selector(".title")
    for tit in titles[21:]:
        title = tit.text
        title_list.append(title)
        
    # 저자 크롤링
    names = driver.find_elements_by_css_selector(".writer")
    for nam in names:
        name = nam.text
        name_list.append(name)

    infos = driver.find_elements_by_css_selector(".etc")
    # 인포
    for inf in infos:
        info = inf.text
        info_spl = info.split(' ')

        # 날짜 크롤링
        for in_sp in info_spl:
            if '20' in in_sp and 'page' not in in_sp and '(' not in in_sp and 'Vol' not in in_sp and 'No' not in in_sp:
                year_list.append(in_sp[0:4])

    # KCI 크롤링
    kcis = driver.find_elements_by_css_selector(".mark > img")
    for kci_num in range(len(kcis)):
        kci_list.append(kcis[kci_num].get_attribute('alt'))

    print(len(title_list), len(name_list), len(year_list))

    # 페이지 클릭
    try:
        driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[5]/a[{0}]'.format(i)).click()
        time.sleep(7)
        if driver.find_elements_by_css_selector(".title")[-1].text == title_list[-1]:
            print("마지막 페이지")
            break
    except:
        print("-")
        break    
    
# driver.close()

100 98 100
140 138 140
마지막 페이지


In [225]:
name_list.append('위치에러-1')

In [226]:
name_list.append('위치에러-2')

In [227]:
len(name_list)

140

In [228]:
df = pd.DataFrame({'저자':name_list, '연도':year_list, '논문제목':title_list})
df

,저자,연도,논문제목
0,"엄미애,방명애",2011,초등 통합학급 교사의 장애학생 교수에 대한 교사효능감과 교수적 수정에 영향을 미치는...
1,"이현주,최나리",2018,중등 예비교사의 통합교육 교수 효능감과 보편적 학습설계에 대한 인식
2,"김주혜 ( Joo Hye Kim ),박은혜 ( Eun Hye Park )",2009,특수학급교사와의 협력을 통한 교사포트폴리오 제작이 통합학급교사의 교수수행능력과 교사...
3,백상수 ( Baek Sang-su ),2017,예비과학교사의 통합교육 태도와 과학 교수효능감 연구
4,"김영한,서영",2013,"통합유치원 교사의 교직문화인식, 교수효능감 및 직무만족도의 관계 분석"
...,...,...,...
135,이은자 ( Eun Ja Lee ),2020,프로그래밍 기반 TPACK 교육 프로그램이 예비 교사의 수업 전문성에 미치는 효과
136,이은자,2020,Effects of Programming-based TPACK Education P...
137,"김보환,장선주,최정실",2014,국어교육 : 외국인 유학생 프레젠테이션 수업 사례 - PBL수업을 중심으로
138,위치에러-1,2014,외국인 유학생 프레젠테이션 수업 사례 - PBL수업을 중심으로


In [229]:
# 엑셀 저장
df.to_excel("riss_%s.xlsx"%keyword, encoding='utf-8-sig')